In [1]:
ls

Untitled.ipynb         sample_submission.csv  train.csv
destinations.csv       test.csv


In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
train=pd.read_csv('train.csv',chunksize=100000)

In [5]:
aggs = []
print('-'*38)
for chunk in train:
    agg = chunk.groupby(['srch_destination_id',
                         'hotel_cluster'])['is_booking'].agg(['sum','count'])
    agg.reset_index(inplace=True)
    aggs.append(agg)
    print('.',end='')

--------------------------------------
.........................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
aggs = pd.concat(aggs, axis=0)

In [7]:
CLICK_WEIGHT = 0.05
agg = aggs.groupby(['srch_destination_id','hotel_cluster']).sum().reset_index()
agg['count'] -= agg['sum']
agg = agg.rename(columns={'sum':'bookings','count':'clicks'})
agg['relevance'] = agg['bookings'] + CLICK_WEIGHT * agg['clicks']
agg.head()

,srch_destination_id,hotel_cluster,bookings,clicks,relevance
0,0,3,0,2,0.10
1,1,20,4,22,5.10
2,1,30,2,20,3.00
3,1,57,0,1,0.05
4,1,60,0,17,0.85


In [8]:
def most_popular(group, n_max=5):
    relevance = group['relevance'].values
    hotel_cluster = group['hotel_cluster'].values
    most_popular = hotel_cluster[np.argsort(relevance)[::-1]][:n_max]
    return np.array_str(most_popular)[1:-1] # remove square brackets

In [9]:
most_pop = agg.groupby(['srch_destination_id']).apply(most_popular)
most_pop = pd.DataFrame(most_pop).rename(columns={0:'hotel_cluster'})

In [11]:
most_pop

,hotel_cluster
srch_destination_id,
0,3
1,20 30 60 57
2,20 30 53 46 41
3,53 60
4,82 25 32 58 78
5,89
6,89
7,42 94 16
8,48 7 43 42 32


In [16]:
test = pd.read_csv('test.csv',
                    dtype={'srch_destination_id':np.int32},
                    usecols=['srch_destination_id'],)

In [17]:
test = test.merge(most_pop, how='left',left_on='srch_destination_id',right_index=True)
test.head(2)

,srch_destination_id,hotel_cluster
0,12243,5 55 37 11 22
1,14474,5


In [18]:
test.hotel_cluster.isnull().sum()


14036

In [19]:

most_pop_all = agg.groupby('hotel_cluster')['relevance'].sum().nlargest(5).index
most_pop_all = np.array_str(most_pop_all)[1:-1]
most_pop_all

'91 48 42 59 28'

In [20]:
test.hotel_cluster.fillna(most_pop_all,inplace=True)

In [21]:
test.hotel_cluster.to_csv('sub.csv',header=True, index_label='id')
